In [ ]:
library(bigrquery)
library(dplyr)
library(tidyr)
library(arules)

# Store the project id
projectid = "ceska-televize"

# Set your query
sql <- "SELECT deviceID as user, eventLabel as porad, count(*) as shednuti from L1.CT_HITS_FACT where eventDate >= '2020-05-01' and eventCategory = 'Video' and eventAction='Prehrat' group by deviceId, eventLabel"

# Run the query and store the data in a dataframe
data <- query_exec(sql, projectid, use_legacy_sql = FALSE, allow_large_results = TRUE, max_pages=Inf)

# Print the query result
nrow(data)
head(data)

user,porad,shlednuti
<chr>,<chr>,<int>
547476995.1585606314,Svět rostlin,231
1748062582.1578690272,Němčina s Bílou paní: Schluss im Schloss,189
1425682668.1588682431,Pozdní jaro: Živočichové a rozkvetlé louky,151
1237462012.1587892788,Josef Jungmann,137
1881005439.1577736071,"Jóga v přírodě: Cviky Hůl, Loď, Volavka",130
1738591574.1589831551,Karel Jaromír Erben: Jeho život a doba,112


In [87]:

# Create a temporary directory

 
# Write our data.frame to a csv
write.csv(data[,c(1,2)], "/tmp/tall_transactions.csv")
 
# Read that csv back in
relations <- read.transactions(
        file = "/tmp/tall_transactions.csv",
        format = "single",
        sep = ",",
        cols=c("user","porad"),
        rm.duplicates = T,
        header=TRUE
)
summary(relations)


transactions as itemMatrix in sparse format with
 86607 rows (elements/itemsets/transactions) and
 15235 columns (items) and a density of 0.0001159044 

most frequent items:
   168 hodin | Neděle 31. května Reportéři ČT | Pondělí 1. června 
                            5426                             3820 
    Události | Pondělí 1. června               Uzly a pomeranče | 
                            2212                             2053 
             Polopatě | Polopatě                          (Other) 
                            1565                           137855 

element (itemset/transaction) length distribution:
sizes
    1     2     3     4     5     6     7     8     9    10    11    12    13 
58289 14796  5830  2957  1691  1010   564   409   259   199   162   109    73 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
   47    31    32    25    26    10    12    10     6     7     4     5     6 
   27    28    29    30    32    33    34    35   

In [88]:
rules <- apriori(relations, parameter = list(support = 0.001, confidence = 0.6))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.6    0.1    1 none FALSE            TRUE       5   0.001      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 86 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[15235 item(s), 86607 transaction(s)] done [0.11s].
sorting and recoding items ... [223 item(s)] done [0.01s].
creating transaction tree ... done [0.05s].
checking subsets of size 1 2 3 4 done [0.00s].
writing ... [75 rule(s)] done [0.00s].
creating S4 object  ... done [0.04s].


In [89]:
inspect(head(rules, n = 50, by = "confidence"))

     lhs                                                rhs                                                 support confidence    coverage      lift count
[1]  {MOST! | 2/8,                                                                                                                                        
      MOST! | 4/8}                                   => {MOST! | 3/8}                                   0.001073816  0.9893617 0.001085363 372.54630    93
[2]  {MOST! | 5/8,                                                                                                                                        
      MOST! | 7/8}                                   => {MOST! | 6/8}                                   0.001039177  0.9890110 0.001050723 426.14565    90
[3]  {MOST! | 6/8,                                                                                                                                        
      MOST! | 8/8}                                   => {MOST! | 7/8} 

In [90]:
library(stringr)

data$serie <- str_split_fixed(data$porad,"\\|",2)[,1]
head(data)


user,porad,shlednuti,serie
<chr>,<chr>,<int>,<chr>
547476995.1585606314,Svět rostlin,231,Svět rostlin
1748062582.1578690272,Němčina s Bílou paní: Schluss im Schloss,189,Němčina s Bílou paní: Schluss im Schloss
1425682668.1588682431,Pozdní jaro: Živočichové a rozkvetlé louky,151,Pozdní jaro: Živočichové a rozkvetlé louky
1237462012.1587892788,Josef Jungmann,137,Josef Jungmann
1881005439.1577736071,"Jóga v přírodě: Cviky Hůl, Loď, Volavka",130,"Jóga v přírodě: Cviky Hůl, Loď, Volavka"
1738591574.1589831551,Karel Jaromír Erben: Jeho život a doba,112,Karel Jaromír Erben: Jeho život a doba


In [91]:


data_serie <- data %>% group_by(user,serie) %>%
  summarise(
     shlednuti=sum(shlednuti)
  ) %>%
  arrange(desc(shlednuti))


head(data_serie)

user,serie,shlednuti
<chr>,<chr>,<int>
547476995.1585606314,Svět rostlin,231
1748062582.1578690272,Němčina s Bílou paní: Schluss im Schloss,189
1527102943.1581135379,Studio 6,172
533469818.1591103377,Černé ovce,169
389682992.1578680241,Černé ovce,158
1425682668.1588682431,Pozdní jaro: Živočichové a rozkvetlé louky,151


In [92]:

 
# Write our data.frame to a csv
write.csv(data_serie[,c(1,2)], "/tmp/tall_serie.csv")
 
# Read that csv back in
relations <- read.transactions(
        file = "/tmp/tall_serie.csv",
        format = "single",
        sep = ",",
        cols=c("user","serie"),
        rm.duplicates = T,
        header=TRUE
)
summary(relations)

transactions as itemMatrix in sparse format with
 86607 rows (elements/itemsets/transactions) and
 3870 columns (items) and a density of 0.0003379697 

most frequent items:
          168 hodin             Události         Reportéři ČT  
                5674                 4375                 4218 
     Vraždy v kruhu  Případy 1. oddělení               (Other) 
                2427                 2293                94290 

element (itemset/transaction) length distribution:
sizes
    1     2     3     4     5     6     7     8     9    10    11    12    13 
71712  9520  2883  1170   533   287   154   111    68    51    35    14    10 
   14    15    16    17    18    19    20    21    22    23    24    25    26 
   12    10    11     6     5     3     1     1     1     1     1     2     2 
   27    29 
    2     1 

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  1.000   1.000   1.000   1.308   1.000  29.000 

includes extended item information - examples:
              labels
1  

In [96]:
rules <- apriori(relations, parameter = list(support = 0.0003, confidence = 0.5))

Apriori

Parameter specification:
 confidence minval smax arem  aval originalSupport maxtime support minlen
        0.5    0.1    1 none FALSE            TRUE       5   3e-04      1
 maxlen target  ext
     10  rules TRUE

Algorithmic control:
 filter tree heap memopt load sort verbose
    0.1 TRUE TRUE  FALSE TRUE    2    TRUE

Absolute minimum support count: 25 

set item appearances ...[0 item(s)] done [0.00s].
set transactions ...[3870 item(s), 86607 transaction(s)] done [0.06s].
sorting and recoding items ... [499 item(s)] done [0.01s].
creating transaction tree ... done [0.06s].
checking subsets of size 1 2 3 4 done [0.00s].
writing ... [21 rule(s)] done [0.00s].
creating S4 object  ... done [0.04s].


In [98]:
inspect(head(rules, n = 50, by = "support"))

     lhs                                       rhs                                         support confidence     coverage        lift count
[1]  {Otázky Václava Moravce ,                                                                                                              
      Reportéři ČT }                        => {168 hodin }                           0.0006235062  0.5684211 0.0010969090    8.676285    54
[2]  {Černé ovce ,                                                                                                                          
      Reportéři ČT }                        => {168 hodin }                           0.0004964957  0.5000000 0.0009929913    7.631918    43
[3]  {Pokusy na magnetické pole}            => {Siločáry magnetu}                     0.0003810316  0.6875000 0.0005542277 1266.857713    33
[4]  {Siločáry magnetu}                     => {Pokusy na magnetické pole}            0.0003810316  0.7021277 0.0005426813 1266.857713    33
[5]  {Čína: D